In [5]:
import csv
import os
import pymongo
from pymongo import MongoClient

In [6]:
def import_csvs():
    try:
        mongo_client = MongoClient("mongodb://localhost:27017/")  # connection
    except Exception as e:
        print(e)  # handle any exceptions that may occur
        return  # exit the function
    db = mongo_client.comunio
    header = ["Playerid", "Datum", "Verein", "Name", "Position", "Punkte", "Punkte pro Spiel", "Passquote", "Zweikampf", "Marktwert"]

    # create an index on the Playerid and Datum fields
    db.raw_player.create_index([('Playerid', 1), ('Datum', 1)])

    for filename in os.listdir('.'):  # loop through all files in the current directory
        if filename.endswith('.csv'):  # check if the file is a CSV file
            try:
                with open(filename, 'r', encoding='utf-8') as csvfile:  # open the CSV file
                    reader = csv.DictReader(csvfile)  # create a DictReader to read the rows

                    for each in reader:  # loop through the rows
                        row = {}
                        for field in header:  # create a dictionary for the current row
                            row[field] = each[field]

                        # replace the document in the collection if it already exists, or insert it if it does not
                        result = db.raw_player.find_one_and_replace(
                            {'Playerid': row['Playerid'], 'Datum': row['Datum']},
                            row,
                            upsert=True
                        )
            except Exception as e:
                print(e)  # handle any exceptions that may occur

import_csvs()  # call the function to import the CSVs